In [1]:
import dataset, model
import albumentations as A
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.nn import functional as F
import numpy as np
import random, os

In [2]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
set_seed(0)

In [3]:
import importlib
importlib.reload(model)
importlib.reload(dataset)

<module 'dataset' from 'D:\\f1\\ch3\\dataset.py'>

In [4]:
import model
import albumentations as A
from albumentations.augmentations.transforms import Flip
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

if __name__ == '__main__':
    ckpt = [ModelCheckpoint(monitor = 'val_mse',save_top_k = 1,mode = 'min'),
           ]
    trainer = Trainer(max_epochs = 85,gpus = 1, callbacks = ckpt, precision=16,deterministic=True,fast_dev_run = False)
    
    train_tr = A.Compose([
        #A.augmentations.transforms.ColorJitter(),
        #A.augmentations.transforms.GaussNoise()
        #A.Resize(32,128)
    ])
    
    val_tr = A.Compose([
        #A.CenterCrop(128,128),
        #A.Resize(32,128)
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':32,'train_tr':train_tr,'val_tr':val_tr})
    
    #ckpt = torch.load('lightning_logs/version_14/checkpoints/epoch=20-step=52499 - Kopie.ckpt')
    #model.load_state_dict(ckpt['state_dict'])
    trainer.fit(model)
    trainer.test(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.

  | Name  | Type | Params
-------------------------------
0 | model | CRNN | 9.5 M 
-------------------------------
9.5 M     Trainable params
0         Non-trainable params
9.5 M     Total params
38.012    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.002244772622361779, 'test_mse': 3247.529052734375}
--------------------------------------------------------------------------------


In [9]:
import model
model = model.Classifier({'lr':3e-4,'batch_size':32,'train_tr':None,'val_tr':None})
    
ckpt = torch.load('') # PATH OF CHECKPOINT
model.load_state_dict(ckpt['state_dict'])
trainer.test(model)
out = trainer.predict(model)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.0021338919177651405, 'test_mse': 3248.539306640625}
--------------------------------------------------------------------------------


Predicting: 1250it [00:00, ?it/s]

In [10]:
import model
concat = torch.tensor([])
for i in range(len(out)):
    concat = torch.cat((concat,torch.tensor(out[i])))
concat = model.toNum(concat).squeeze()

In [11]:
import model
model.writeSub(concat) 

In [12]:
!aicrowd submission create -c f1-speed-recognition -f submission.csv

submission.csv ------------------- 100.0% • 105.1/103.5 KB • 1.5 MB/s • 0:00:00                                                   +-------------------------+                                                    
                                                   | Successfully submitted! |                                                    
                                                   +-------------------------+                                                    
                                                         Important links                                                          
+--------------------------------------------------------------------------------------------------------------------------------+
|  This submission | https://www.aicrowd.com/challenges/ai-blitz-8/problems/f1-speed-recognition/submissions/137250              |
|                  |                                                                                                             |
|  A